In [140]:
from datasets import load_dataset
language = 'en'

dataset = load_dataset('wikiann','en')
dataset["train"][2]

Found cached dataset wikiann (/home/s6amalia/.cache/huggingface/datasets/wikiann/en/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e)


  0%|          | 0/3 [00:00<?, ?it/s]

{'tokens': ['Karl', 'Ove', 'Knausgård', '(', 'born', '1968', ')'],
 'ner_tags': [1, 2, 2, 0, 0, 0, 0],
 'langs': ['en', 'en', 'en', 'en', 'en', 'en', 'en'],
 'spans': ['PER: Karl Ove Knausgård']}

In [141]:
dataset

DatasetDict({
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 10000
    })
    train: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 20000
    })
})

In [142]:
label_list = dataset["train"].features[f"ner_tags"].feature.names
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

In [143]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [144]:
example = dataset["train"][2]
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

['[CLS]',
 'karl',
 'o',
 '##ve',
 'kn',
 '##aus',
 '##gard',
 '(',
 'born',
 '1968',
 ')',
 '[SEP]']

In [145]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [146]:
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

Loading cached processed dataset at /home/s6amalia/.cache/huggingface/datasets/wikiann/en/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e/cache-7431a44ae42d950d.arrow


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Loading cached processed dataset at /home/s6amalia/.cache/huggingface/datasets/wikiann/en/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e/cache-35796646de0bdc3d.arrow


In [147]:
tokenized_inputs = tokenizer(example["tokens"], truncation=True, is_split_into_words=True)
word_ids = tokenized_inputs.word_ids(batch_index=0)
word_ids

[None, 0, 1, 1, 2, 2, 2, 3, 4, 5, 6, None]

In [148]:
tokens = tokenizer.convert_ids_to_tokens(tokenized_dataset['test']["input_ids"][0])
tokenized_dataset['test']["tokens"][0].word_ids(batch_index=0)

AttributeError: 'list' object has no attribute 'word_ids'

In [149]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [150]:
import evaluate

seqeval = evaluate.load("seqeval")

In [151]:
import numpy as np

labels = [label_list[i] for i in example[f"ner_tags"]]


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [152]:
labels = [label_list[i] for i in example[f"ner_tags"]]
labels

['B-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O']

In [153]:

id2label = {
    0: "O",
    1: 'B-PER',
    2: 'I-PER',
    3: 'B-ORG',
    4: 'I-ORG',
    5: 'B-LOC',
    6: 'I-LOC',
}
label2id = {
    "O": 0,
    'B-PER': 1,
    'I-PER': 2,
    'B-ORG': 3,
    'I-ORG': 4,
    'B-LOC': 5,
    'I-LOC': 6,
}

In [16]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    'xlm-roberta-base', num_labels=7, id2label=id2label, label2id=label2id
)

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="./wikiann_fine_tuned",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="no",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

In [188]:

text=' '.join(dataset["test"]['tokens'][2])
text

'Blacktown railway station'

In [155]:
from transformers import pipeline
classifier = pipeline("ner", model="/home/s6amalia/xlmroberta-wikiann-en.pt", tokenizer = 'xlm-roberta-base')

classifier(' '.join(dataset["test"]['tokens'][0]))

[{'entity': 'B-LOC',
  'score': 0.98101395,
  'index': 18,
  'word': '▁India',
  'start': 68,
  'end': 73},
 {'entity': 'B-LOC',
  'score': 0.9404207,
  'index': 23,
  'word': '▁Ad',
  'start': 90,
  'end': 92},
 {'entity': 'I-LOC',
  'score': 0.8774265,
  'index': 24,
  'word': 'yar',
  'start': 92,
  'end': 95}]

In [206]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
inputs = tokenizer(dataset["test"]['tokens'][3],truncation=True, is_split_into_words=True,return_tensors="pt")

In [207]:
from transformers import AutoModelForTokenClassification
import torch
model = AutoModelForTokenClassification.from_pretrained("/home/s6amalia/xlmroberta-wikiann-en.pt")
with torch.no_grad():
    logits = model(**inputs).logits

In [208]:
predictions = torch.argmax(logits, dim=-1)
predicted_token_class = [model.config.id2label[t.item()] for t in predictions[0]]
# (predicted_token_class)

In [234]:
import pandas as pd
res=pd.DataFrame([inputs.word_ids(batch_index=0),predictions[0].tolist(),tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])]
)
# tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
res=res.transpose()
res.columns=['idx','label','word']
res

,idx,label,word
0,None,0,<s>
1,0,0,▁''
2,1,5,▁My
3,1,6,cale
4,1,6,sis
5,2,6,▁perse
6,2,6,us
7,3,6,▁la
8,3,6,las
9,3,6,sis


In [236]:
res=res.drop_duplicates(subset=['idx'], keep='first')[1:]
res['true_label']  = dataset["test"][3]['ner_tags']

,idx,label,word,true_label
1,False,False,True,False
2,True,True,True,True
5,True,True,True,True
7,True,True,True,True
10,True,False,True,False
11,True,False,True,False
12,True,False,True,False
15,True,False,True,False
17,True,False,True,False
18,True,False,True,False


In [211]:
' '.join(dataset["test"]['tokens'][3])

"'' Mycalesis perseus lalassis '' ( Hewitson , 1864 )"

In [225]:
tokenized_dataset['test']['labels'][3]

22

In [227]:
dataset["test"][3]['ner_tags']

[0, 5, 6, 6, 0, 0, 0, 0, 0, 0]